In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

import _pickle as pickle
import random
import numpy as np

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# torch.cuda.set_device(2)

cpu


In [9]:
from Model import *

In [15]:
def pad_data(data, max_sen_len=50, max_doc_len=90, padding_idx=400003):
    lines = []
    lengths = []
    for doc in data:
        lines.append([])
        lengths.append([])
        for line in doc[2][0:max_doc_len]:
            if len(line) > max_sen_len:
                line = line[0:max_sen_len]
                lengths[-1].append(max_sen_len)
            else:
                lengths[-1].append(len(line))
                line = line + [padding_idx for _ in range(max_sen_len - len(line))]
            lines[-1].append(line)
    return lines, lengths

In [16]:
def pad_titles(data, max_sen_len=20, padding_idx=400003):
    lines = []
    for row in data:
        line = row[0]
        if len(line) > max_sen_len:
            lines.append(line[0:max_sen_len])
        else:
            lines.append(line + [padding_idx for _ in range(max_sen_len - len(line))])
    return lines

In [20]:
class_mapping = {
    'left': [1, 0, 0, 0, 0],
    'left-center': [0, 1, 0, 0, 0],
    'least': [0, 0, 1, 0, 0],
    'right-center': [0, 0, 0, 1, 0],
    'right': [0, 0, 0, 0, 1],
}

EMB = pickle.load(open('../../glove.6B/100d.txt', 'rb'))
EMB.append(np.zeros(100))
EMB = torch.Tensor(EMB)

URLS = pickle.load(open('./Meta/urls', 'rb'))
NO_URLS = len(URLS)

UnpicklingError: could not find MARK

In [12]:
class Train():
    def __init__(self, model):
        self.model = model
        self.raw_data, self.lines, self.pad_lengths = None, None, None 
        self.truth, self.class_, self.urls = None, None, None
        self.file_no = 0
        self.max_file_no = 8
        self.batch_size = 20
        self.loss1 = nn.BCEWithLogitsLoss()
        self.loss2 = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters())
        self.losses = []
        
    def __get_next_batch(self):
        pass
    
    def _load_next(self):
        self.raw_data = pickle.load(open('training_' + str(self.file_no), 'rb'))
        self.lines, self.pad_lengths = pad_data(self.raw_data)
        self.class = [class_mapping[row[-1]] for row in self.raw_data]
        self.truth = [int(row[-2]) for row in self.raw_data]
        self.lengths = [[] for _ in range(90)]
        for i, j in enumerate(self.lines):
            if not len(j) == 0:
                self.lengths[len(j)-1].append(i)
        self.urls = [i[i][0:90] for i in self.raw_data]
        self.batches = []
        for unit in self.lengths:
            for i in range(0,len(unit), self.batch_size):
                self.batches.append(unit[i:i+self.batch_size])
        self.titles = pad_titles(self.raw_data)
        random.shuffle(self.batches)
        self.file_no += 1
    
    

SyntaxError: invalid syntax (<ipython-input-12-6d6c44ac06b1>, line 20)